# Quinta Actividad: Cargando archivo CSV: 

- **Data source**: [Cantidad de ambulancias, camas y salas (consideradas trazadoras) por departamento, año y naturaleza jurídica para servicios habilitados](https://www.datos.gov.co/Salud-y-Protecci-n-Social/Cantidad-de-ambulancias-camas-y-salas-consideradas/fa2g-cdft/about_data)
- **Introducción**: Seleccioné este dataset para desarrollar mi proyecto, trataré de analizar una problemática como lo es el la cantidad de salas y camas disponibles en hospitales, así como practicar los conocimientos adquiridos en clase, sobre la creación de gráficos.
- **Objetivo principal**: El objetivo principal de esta entrega es crear instancia de conexión a una base de datos de PostgreSQL y dejar evidencia de las consultas.

# 0. Librerias

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sqlalchemy import create_engine
from sqlalchemy import text
import psycopg2
from dotenv import load_dotenv
import os
ruta = os.getcwd()

# 1. Conection to the DataBase

In [3]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener las variables de entorno
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_port = os.getenv('DB_PORT')


# Conexión a la base de datos
connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)
connection = engine.connect()

# 2. Data Extraction and Loading

In [4]:
#Se lee el archivo CSV
df = pd.read_csv(os.path.join(ruta, 'Cantidad_de_ambulancias__camas_y_salas__consideradas_trazadoras__por_departamento__a_o_y_naturaleza_jur_dica_para_servicios_habilitados_20240805.csv'))
#Se envía el DataFrame a postgres y se crea la tabla automáticamente
df.to_sql('salas_y_camas', engine, if_exists='replace', index=False)

378

# 3. Data loading

In [5]:
#Función para consultar en el DB y se imprima el resultado en python
def runQuery (sql):
    result = engine.connect().execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())

In [6]:
#Imprimir toda la tabla del DB
query = '''
SELECT * FROM salas_y_camas
'''
runQuery(query)

,Departamento,Año,Naturaleza jurídica,AMBULANCIAS Básica,AMBULANCIAS Medicalizada,CAMAS Adultos,CAMAS Cuidado Agudo Mental,CAMAS Cuidado Intensivo Adulto,CAMAS Cuidado Intensivo Neonatal,CAMAS Cuidado Intensivo Pediátrico,...,CAMAS Cuidado Intermedio Neonatal,CAMAS Cuidado Intermedio Pediátrico,CAMAS Farmacodependencia,CAMAS Obstetricia,CAMAS Pediátrica,CAMAS Psiquiatría,CAMAS Unidad de Quemados Adulto,CAMAS Unidad de Quemados Pediátrico,SALAS Partos,SALAS Quirófano
0,Amazonas,2017.0,Privada,0,0,41,0,0,0,0,...,0,0,0,2,7,0,0,0,0,1
1,Amazonas,2017.0,Pública,3,0,27,0,0,0,0,...,0,3,0,21,28,0,0,0,4,2
2,Amazonas,2018.0,Privada,0,0,41,0,0,0,0,...,0,0,30,2,7,0,0,0,0,1
3,Amazonas,2018.0,Pública,3,0,27,0,0,0,0,...,0,3,0,21,28,0,0,0,8,2
4,Amazonas,2019.0,Privada,0,0,41,0,0,0,0,...,0,0,30,2,7,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,Valle del cauca,2022.0,Pública,133,9,491,0,45,0,0,...,0,0,0,0,168,0,0,0,44,0
374,Vaupés,2022.0,Privada,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375,Vaupés,2022.0,Pública,2,0,14,0,0,0,0,...,0,0,0,1,8,0,0,0,3,0
376,Vichada,2022.0,Privada,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
query_0 = '''
SELECT 
    Departamento, 
    año,
    AVG("AMBULANCIAS Básica") AS Prom_Ambuancias_Basicas,
    AVG("AMBULANCIAS Medicalizadas") AS  Prom_Ambulancias_Medicalizadas,
    (
        (COALESCE("AMBULANCIAS Básica") + COALESCE("AMBULANCIAS Medicalizadas")
    )/2.0 AS Prom_Ambulancias
FROM 
    salas_y_camas
GROUP BY 
    Departamento, año;
'''
runQuery(query_0)

ProgrammingError: (psycopg2.errors.SyntaxError) error de sintaxis en o cerca de «AS»
LINE 9:     )/2.0 AS Prom_Ambulancias
                  ^

[SQL: 
SELECT 
    Departamento, 
    año,
    AVG("AMBULANCIAS Básica") AS Prom_Ambuancias_Basicas,
    AVG("AMBULANCIAS Medicalizadas") AS  Prom_Ambulancias_Medicalizadas,
    (
        (COALESCE("AMBULANCIAS Básica") + COALESCE("AMBULANCIAS Medicalizadas")
    )/2.0 AS Prom_Ambulancias
FROM 
    salas_y_camas
GROUP BY 
    Departamento, año;
]
(Background on this error at: https://sqlalche.me/e/20/f405)